In [ ]:
#!/usr/bin/env python3
import pympi as pmp
import os

class EafFile:
    def _getParticipants(self):
        self.participants = []

        for k in self.eafFile.tiers.keys():
            if 'PARTICIPANT' in self.eafFile.tiers[k][2]:
                participant = self.eafFile.tiers[k][2]['PARTICIPANT']
                if participant not in self.participants:
                    self.participants.append(participant)
            elif self.eafFile.tiers[k][2]["TIER_ID"] == "code":
                participant = "code"
                if participant not in self.participants:
                    self.participants.append(participant)

    def _getTiers(self):
        self.tiers = []

        for k in self.eafFile.tiers.keys():
            if 'TIER_ID' in self.eafFile.tiers[k][2]:
                tier_id = self.eafFile.tiers[k][2]['TIER_ID']
                if tier_id not in self.tiers:
                    self.tiers.append(tier_id)

    def __init__(self, eafFilePath):
        self.eafPath = eafFilePath
        self.eafFile = pmp.Elan.Eaf(eafFilePath)

        self._getParticipants()
        self._getTiers()

    def toRTTM(self, path_to_write_rttm):
        """
        Función para escribir un nuevo archivo .rttm que es una transcripción del archivo .eaf dado como entrada.
        """
        sampling_freq = 1000

        base = os.path.basename(self.eafPath)
        name = os.path.splitext(base)[0]

        print('Procesando archivo: {}'.format(name))

        with open(os.path.join(path_to_write_rttm, name + ".rttm"), "w", encoding='utf-8') as rttm:
            for participant in self.participants:
                if participant in self.eafFile.tiers:
                    for _, val in self.eafFile.tiers[participant][0].items():
                        start = val[0]
                        end = val[1]

                        t0 = self.eafFile.timeslots[start] / sampling_freq
                        length = self.eafFile.timeslots[end] / sampling_freq - t0

                        rttm.write(u"SPEAKER\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(
                            name, 1, "%.2f" % t0, "%.2f" % length, "<NA>", "<NA>", participant, 1
                        ))

In [2]:
# Uso del código

# Directorio raíz donde buscar archivos .eaf
root_dir = "transcripciones-segunda-vuelta/random"  # Reemplaza esto con la ruta a tu directorio raíz

# Recorre todos los subdirectorios buscando archivos .eaf
for subdir, dirs, files in os.walk(root_dir):
    for file in files:
        if file.endswith('.eaf'):
            eaf_file_path = os.path.join(subdir, file)
            path_to_write_rttm = subdir  # Guarda en el mismo directorio que el archivo .eaf
            eaf_instance = EafFile(eaf_file_path)
            eaf_instance.toRTTM(path_to_write_rttm)

Listo: se generaron los RTTM.
